In [1]:
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import alpaca_trade_api as api
import yaml

In [2]:
# Read config
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [3]:
# Create Alpaca client
alpaca = api.REST(config['api-key'], config['api-secret'])

In [4]:
year = 2023
month = 6
day = 1
hour = 12

start_minute = 0
end_minute = 30
total_minutes = end_minute - start_minute
start = f'{year:02}-{month:02}-{day:02}T{hour:02}:{start_minute:02}:00Z'

# trades data fetches one minute less than bars, we have to specify one minute more for trades
end_open = f'{year:02}-{month:02}-{day:02}T{hour:02}:{end_minute:02}:00Z'
end_close = f'{year:02}-{month:02}-{day:02}T{hour:02}:{end_minute + 1:02}:00Z'

tick_data = alpaca.get_trades('AAPL', start=start, end=end_close).df
minute_data = alpaca.get_bars('AAPL', timeframe="1Min", start=start, end=end_open).df

In [5]:
tick_data

,conditions,id,price,size,exchange,tape
timestamp,,,,,,
2023-06-01 12:00:00.023700883+00:00,"[@, T, I]",1,177.7600,1,D,C
2023-06-01 12:00:00.033288239+00:00,"[@, T]",2,177.8403,2273,D,C
2023-06-01 12:00:00.039942148+00:00,"[@, T]",3,177.8111,200,D,C
2023-06-01 12:00:00.047837206+00:00,"[@, T]",4,177.8400,200,D,C
2023-06-01 12:00:00.048880992+00:00,"[@, T, I]",5,177.8991,30,D,C
...,...,...,...,...,...,...
2023-06-01 12:30:55.011554743+00:00,"[@, T, I]",1389,177.7150,1,D,C
2023-06-01 12:30:59.799966363+00:00,"[@, F, T, I]",2287,177.7900,2,P,C
2023-06-01 12:30:59.799966461+00:00,"[@, F, T, I]",2288,177.8000,98,P,C


In [6]:
aggregated_minute_data = pd.DataFrame(
    columns=['close', 'high', 'low', 'trade_count', 'open', 'volume'],
    index=pd.date_range(start=start, end=end_open,freq='1min'))

for i in range(total_minutes + 1):
    
    minute_start = f'{year:02}-{month:02}-{day:02}T{hour:02}:{start_minute + i:02}:00Z'
    minute_end = f'{year:02}-{month:02}-{day:02}T{hour:02}:{start_minute + i + 1:02}:00Z'

    # all data past the starting minute
    data_after_start = tick_data[tick_data.index >= minute_start]
    
    # all data past the starting minute and before the ending minute
    data = data_after_start[data_after_start.index < minute_end]

    aggregated_row = {
        'low': data['price'].min(),
        'high': data['price'].max(),
        'open': data['price'].iloc[0],
        'close': data['price'].iloc[-1],
        'volume': data['size'].sum(),
        'trade_count': data.shape[0]
    }

    aggregated_minute_data.loc[minute_start] = aggregated_row

In [7]:
real = minute_data.drop(columns=['vwap'])
real

,close,high,low,trade_count,open,volume
timestamp,,,,,,
2023-06-01 12:00:00+00:00,177.8000,178.1800,177.6900,527,177.8403,54884
2023-06-01 12:01:00+00:00,177.7900,177.8800,177.0600,1103,177.7600,11915
2023-06-01 12:02:00+00:00,177.4900,177.8400,177.4900,400,177.8300,3651
2023-06-01 12:03:00+00:00,177.8300,177.8600,177.8300,85,177.8600,8902
2023-06-01 12:04:00+00:00,177.8200,177.8500,177.8200,46,177.8300,3377
2023-06-01 12:05:00+00:00,177.8100,177.8100,177.8000,35,177.8000,4988
2023-06-01 12:06:00+00:00,177.8200,177.8200,177.8000,27,177.8000,948
2023-06-01 12:07:00+00:00,177.8000,177.8000,177.7998,19,177.7998,1111
2023-06-01 12:08:00+00:00,177.8194,177.8200,177.7600,30,177.7620,2837


In [8]:
aggr = aggregated_minute_data
aggr

,close,high,low,trade_count,open,volume
2023-06-01 12:00:00+00:00,177.8,178.18,177.25,527.0,177.76,54884.0
2023-06-01 12:01:00+00:00,177.49,179.43,177.01,1103.0,177.97,11915.0
2023-06-01 12:02:00+00:00,177.68,178.0,177.01,400.0,177.25,3651.0
2023-06-01 12:03:00+00:00,177.83,177.925,177.8,85.0,177.825,8902.0
2023-06-01 12:04:00+00:00,177.81,177.85,177.8,46.0,177.82,3377.0
2023-06-01 12:05:00+00:00,177.81,177.82,177.25,35.0,177.8,4988.0
2023-06-01 12:06:00+00:00,177.79,177.82,177.76,27.0,177.8,948.0
2023-06-01 12:07:00+00:00,177.76,177.8,177.76,19.0,177.8,1111.0
2023-06-01 12:08:00+00:00,177.8194,177.82,177.76,30.0,177.76,2837.0
2023-06-01 12:09:00+00:00,177.69,177.82,177.69,60.0,177.82,2138.0
